In [1]:
import os
from tifffile import imread, imwrite
import numpy as np

file_path = r'C:\Users\va332845\OneDrive - Knights - University of Central Florida\Nanoscopy Group\Experiments\STED\Collaboration with Stephen Jacobe\Data_Vahid\2022_08_17_HeLa\2 sec delay between images\images'
save_path = r'C:\Users\va332845\OneDrive - Knights - University of Central Florida\Nanoscopy Group\Experiments\STED\Collaboration with Stephen Jacobe\Data_Vahid\2022_08_17_HeLa\2 sec delay between images\patches'
list_files = os.listdir(file_path)
M = len(list_files)
patch_size = 256
threshold = 0.0
count1 = 0
for i in list_files:
    f = imread(os.path.join(file_path, '', i)).astype(np.float64)
    f = f / f.max()
    f = f - f.min()
    f = f / f.max()
    f[f < 0] = 0
    w = f.shape[-1]
    h = f.shape[-2]
    if len(f.shape) == 3:
        t = f.shape[0]
    else:
        t = 1
        f = f.reshape((1, f.shape[0], f.shape[1]))
    nx = int(np.ceil(h / patch_size))
    ny = int(np.ceil(w / patch_size))
    
    x = np.zeros((nx * ny * t, patch_size, patch_size))
    nnx = np.floor(np.linspace(0, h - patch_size, nx)).astype(np.int32)
    nny = np.floor(np.linspace(0, w - patch_size, ny)).astype(np.int32)
    print(nnx,nny)
    count = 0
    for p in range(nx):
        for q in range(ny):
            for j in range(t):
                x[count, :, :] = f[j, nnx[p]:nnx[p] + patch_size, nny[q]:nny[q] + patch_size]
                count = count + 1

    norm_x = np.linalg.norm(x, axis=(1, 2))
    norm_x = norm_x / norm_x.max()
    ind_norm = np.where(norm_x > threshold)[0]
    y = np.zeros((len(ind_norm), patch_size, patch_size))
    for k in range(len(ind_norm)):
        y[k] = x[ind_norm[k]]
    y = np.uint16(y * (2 ** 16 - 1))
    imwrite(os.path.join(save_path, '', str(count1)+'.tif'), y, imagej=True, metadata={'axes': 'TYX'})
    count1 = count1+1


[  0 244] [ 0 77]
[  0 172 344] [  0 145]
[  0 244] [ 0 77]
[  0 244] [ 0 78]
[  0 244] [ 0 77]
